# This code measures how fast it can stream full events from Elasticsearch

In [1]:
from elasticsearch import Elasticsearch
from elasticsearch.helpers import scan
from time import time

es = Elasticsearch(['atlas-kibana.mwt2.org:9200'],timeout=60)

## Select data format you would like to stream

In [2]:
my_index = "data15_13tev_aod"

### Here a stream everything query

In [3]:
my_query = {
        "query" : {
                "match_all": {}
        }
}

### actually streaming events

In [6]:
t0 = time()

scroll = scan(es, query=my_query, index=my_index, scroll='5m', timeout="5m", size=100)

count = 0
for res in scroll:
    if not count%1000: print(count)
    count += 1
    
t1 = time()

print (count, 'events in %f' %(t1-t0))

0
100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
2100
2200
2300
2400


KeyboardInterrupt: 